# Modelado con LSTM usando landmarks de MediaPipe

Este notebook carga los puntos clave (landmarks) extraídos en el preprocesamiento y entrena un modelo LSTM para clasificar los números en lenguaje de señas.

In [ ]:
# Librerías necesarias
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

## 1. Cargar los landmarks y preparar los datos

In [ ]:
# Ruta al archivo CSV de landmarks
landmarks_csv = os.path.join("preprocessed_numbers", "landmarks.csv")
df = pd.read_csv(landmarks_csv)

# Solo usamos las clases numéricas (0-9)
df = df[df["class"].isin([str(i) for i in range(10)])].reset_index(drop=True)

# Extraer X (landmarks) y y (clase)
X = df[[f"lm_{i}" for i in range(63)]].values.astype(np.float32)
y = df["class"].values

# Codificar etiquetas
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Para LSTM: reshape a (samples, timesteps, features)
# MediaPipe Hands tiene 21 puntos, cada uno con (x, y, z)
X_lstm = X.reshape(-1, 21, 3)
print("Shape X_lstm:", X_lstm.shape)
print("Shape y_enc:", y_enc.shape)

## 2. Dividir en entrenamiento y validación

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_lstm, y_enc, test_size=0.15, random_state=42, stratify=y_enc
)
print("Train:", X_train.shape, "Val:", X_val.shape)

## 3. Construir el modelo LSTM

In [ ]:
num_classes = len(le.classes_)

model = keras.Sequential([
    keras.layers.Input(shape=(21, 3)),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

## 4. Entrenar el modelo

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=32,
    callbacks=callbacks
)

## 5. Evaluar y guardar el modelo

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validación - Loss: {val_loss:.4f}  Acc: {val_acc:.4f}")

# Guardar modelo y codificador de etiquetas en carpeta Modelos
import os
os.makedirs("Modelos", exist_ok=True)
model.save(os.path.join("Modelos", "modelo_lstm_landmarks.keras"))
import joblib
joblib.dump(le, os.path.join("Modelos", "label_encoder_landmarks.pkl"))

¡Listo! El modelo LSTM entrenado y el codificador de etiquetas han sido guardados en la carpeta `Modelos`.